In [1]:
# -*- coding: utf-8 -*-
import matplotlib.pyplot as plt
import matplotlib
import math
import datetime
import networkx as nx
import os

In [2]:
def TodaysDate():
        
    Today = datetime.date.today()
    TodaysDate = Today.strftime('%d%b%Y')
    
    return TodaysDate

In [3]:
def DNAcoding_sequence(DNASequence, QualityScoreSequence, start_sequence, stop_sequence):
#utilises ONLY ONE stop_sequence, returns ONLY ONE coding_sequence
    
    QualityScoreString = """!"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~"""
    ThresholdQualityScore = 29 # ThresholdQualityScore must be between 0 and 93
    ThresholdQualityString = QualityScoreString[ThresholdQualityScore:]
    
    MinLength = 24
    MaxLength = 240
            
    StartIndex = DNASequence.find(start_sequence) + len(start_sequence)
    StopIndex = DNASequence.rfind(stop_sequence)
    coding_sequence =  DNASequence[StartIndex:StopIndex]
    if MinLength <= len(coding_sequence) and len(coding_sequence) <= MaxLength and len(coding_sequence)%3 == 0:
        for Character in QualityScoreSequence[StartIndex:StopIndex]:
            if Character not in ThresholdQualityString:
                return None
        return str(coding_sequence)

In [4]:
def Translation(coding_sequence):
#translates DNA sequence

    TranslationCode = {
                    'AAA':'K','AAC':'N','AAG':'K','AAU':'N',
                    'ACA':'T','ACC':'T','ACG':'T','ACU':'T',
                    'AGA':'R','AGC':'S','AGG':'R','AGU':'S',
                    'AUA':'I','AUC':'I','AUG':'M','AUU':'I',
                    
                    'CAA':'Q','CAC':'H','CAG':'Q','CAU':'H',
                    'CCA':'P','CCC':'P','CCG':'P','CCU':'P',
                    'CGA':'R','CGC':'R','CGG':'R','CGU':'R',
                    'CUA':'L','CUC':'L','CUG':'L','CUU':'L',
                    
                    'GAA':'E','GAC':'D','GAG':'E','GAU':'D',
                    'GCA':'A','GCC':'A','GCG':'A','GCU':'A',
                    'GGA':'G','GGC':'G','GGG':'G','GGU':'G',
                    'GUA':'V','GUC':'V','GUG':'V','GUU':'V',
                    
                    'UAA':'#','UAC':'Y','UAG':'*','UAU':'Y',
                    'UCA':'S','UCC':'S','UCG':'S','UCU':'S',
                    'UGA':'&','UGC':'C','UGG':'W','UGU':'C',
                    'UUA':'L','UUC':'F','UUG':'L','UUU':'F'
                        }
    # UAA (ochre) — #
    # UAG (amber) — *
    # UGA (opal) — &
                    
    TranscriptionCode = {'A':'A','C':'C','G':'G','T':'U','U':'T'}
      
    RNASequence = ''
    for Nucleotide in coding_sequence:
        RNASequence += TranscriptionCode.get(Nucleotide,'X')
    #converts DNA to RNA
    #print RNASequence
        
    peptide = ''
    while len(RNASequence) != 0:
        peptide += TranslationCode.get(RNASequence[0:3],'Do not fuck with me!')
        RNASequence = RNASequence[3:]
    return peptide

In [5]:
def SingleSelectionRoundSummary(fastq_file_path):
#returns a list of lists with peptide-sequences and their frequencies, sorted by frequency in descending order
    
    RawDataFile = open(fastq_file_path, 'r')
    lines = RawDataFile.readlines()
    RawDataFile.close
    
    #start_sequence = 'ATG' # Met codon
    #stop_sequence = 'TGCGGCAGC'# Akane seams to have trimmed siquences
    #stop_sequence = 'TAG' # amber stop codon
    
    start_sequence = 'TAATACGACTCACTATAGGGTTAACTTTAAGAAGGAGATATACATATG'    # NNK - T7g10M.F48 
    stop_sequence = 'TGCGGCAGCGGCAGCGGCAGCTAGGACGGGGGGCGGAAA' #NNK - CGS3an13.R39 
    #start_sequence = 'TAATACGACTCACTATAGGGTTGAACTTTAAGTAGGAGATATATCCATG'   #NNU - T7-CH-F49
    #stop_sequence = 'TGTGGGTCTGGGTCTGGGTCTTAGGACGGGGGGCGGAAA'  #NNU - CGS3-CH-R39
    
    SingleSelectionRoundSummary = {}
    #creates empty SingleSelectionRoundSummary dictionary to store the results from a single round of selection
    #SingleSelectionRoundSummary = {peptideY:    {coding_sequence_YZ:    Occurrence_YZ}}
        
    #populates SingleSelectionRoundSummary dictionary with the results from a single round of selection
    for i,line in enumerate(lines):
        if start_sequence in line and stop_sequence in line:
            coding_sequence = DNAcoding_sequence(line, lines[i + 2], start_sequence, stop_sequence)
            if coding_sequence != None:
                peptide = Translation(coding_sequence)
                if peptide not in SingleSelectionRoundSummary:
                    SingleSelectionRoundSummary[str(peptide)] = {str(coding_sequence) : 1}
                else:
                    if coding_sequence not in SingleSelectionRoundSummary[str(peptide)]:
                        SingleSelectionRoundSummary[str(peptide)][str(coding_sequence)] = 1
                    else:
                        SingleSelectionRoundSummary[str(peptide)][str(coding_sequence)] += 1

    return SingleSelectionRoundSummary

In [6]:
def HammingDistance(Sequence1, Sequence2):
    
    if len(Sequence1) < len(Sequence2):
        Sequence1 = Sequence1 + (len(Sequence2) - len(Sequence1)) * '%'
    elif len(Sequence1) > len(Sequence2):
        Sequence2 = Sequence2 + (len(Sequence1) - len(Sequence2)) * '%'
    
    HammingDistance = 0
    for i in range(len(Sequence1)):
        if Sequence1[i] == Sequence2[i]:
            HammingDistance = HammingDistance
        else:
            HammingDistance = HammingDistance + 1
            
    return HammingDistance

In [7]:
def HammingDistanceBasedFormating(Sequence1, Sequence2):
    
    if len(Sequence1) < len(Sequence2):
        Sequence1 = Sequence1 + (len(Sequence2) - len(Sequence1)) * '.'
    elif len(Sequence1) > len(Sequence2):
        Sequence2 = Sequence2 + (len(Sequence1) - len(Sequence2)) * '.'
    
    HammingDistance = 0
    FormatedSequence2 = ''
    for i in range(len(Sequence1)):
        if Sequence1[i] == Sequence2[i]:
            FormatedSequence2 += Sequence2[i].lower()
            HammingDistance = HammingDistance
        else:
            FormatedSequence2 += Sequence2[i]
            HammingDistance = HammingDistance + 1            
    return FormatedSequence2

In [8]:
def Completedisplay_summary(data_directory_path):
# returns a display_summary dictionary with the following structure
# display_summary = {SelectionRound_X:    {peptideXY:    {CodingDNA_XYZ:    Occurrence_XYZ}}}

    Completedisplay_summary = {}
    # creates empty display_summary dictionary to store the results from all the rounds of selection

    for file in os.listdir(data_directory_path):
        
        file_path = os.path.join(data_directory_path, file)
          
        if file.endswith('.fastq'): # this conditional is necessary; without it some shit appears in the beginning of the file list
            cycle_numberFirstDigit = file[file.find('.')-2]
            cycle_numberSecondDigit = file[file.find('.')-1]
            if cycle_numberFirstDigit == '0':
                cycle_number = int(cycle_numberSecondDigit)
                #print cycle_number
            elif cycle_numberFirstDigit != '0':
                cycle_number = int(file[file.find('.')-2 : file.find('.')])
                #print cycle_number
        #(1.A) extracts the round number from the file name (file name should have two digit number before full stop — '00.') 
                
            SelectionRoundSummary = SingleSelectionRoundSummary(file_path)
            #(1.B) extracts single round results 
                    
            Completedisplay_summary[cycle_number] = SelectionRoundSummary
            #(1.C) populate ConcatenatedResultsList
            #print ConcatenatedResultsList
            
    return Completedisplay_summary

In [9]:
def peptidesOccurrences_BY_Round(data_directory_path):
    display_summary = Completedisplay_summary(data_directory_path)
    
    peptidesOccurrences_BY_Round = {}
    for Round in display_summary:
        peptidesOccurrences_IN_Round = {}
        for peptide in display_summary[Round]:
            peptidesOccurrences_IN_Round[peptide] = sum(display_summary[Round][peptide].values())
        peptidesOccurrences_BY_Round[Round] = peptidesOccurrences_IN_Round
        
    return peptidesOccurrences_BY_Round

In [10]:
def DNAsOccurrences_BY_Round(data_directory_path):
    display_summary = Completedisplay_summary(data_directory_path)
    
    DNAsOccurrences_BY_Round = {}
    for Round in display_summary:
        DNAsOccurrences_IN_Round = {}
        for peptide in display_summary[Round]:
            for DNA in display_summary[Round][peptide]:
                DNAsOccurrences_IN_Round[DNA] = display_summary[Round][peptide][DNA]
        DNAsOccurrences_BY_Round[Round] = DNAsOccurrences_IN_Round

    return DNAsOccurrences_BY_Round

In [11]:
def TotalReads_BY_Round(data_directory_path):
    display_summary = Completedisplay_summary(data_directory_path)
    peptides_BY_Round = peptidesOccurrences_BY_Round(data_directory_path)
    
    TotalReads_BY_Round = {}
    for Round in display_summary:
        TotalReads_BY_Round[Round] = sum(peptides_BY_Round[Round].values())
        
    return TotalReads_BY_Round

In [12]:
def BaseRoundSortedpeptidesList(data_directory_path, base_cycle):
    peptides_BY_Round = peptidesOccurrences_BY_Round(data_directory_path)  
            
    peptidesOccurrencesInBaseRound = peptides_BY_Round[base_cycle]
    BaseRoundSortedpeptidesList = sorted(peptidesOccurrencesInBaseRound, key = peptidesOccurrencesInBaseRound.get, reverse = True)
    
    return BaseRoundSortedpeptidesList

In [13]:
def BaseRoundSortedDNAsList(data_directory_path, base_cycle):
    DNAs_BY_Round = DNAsOccurrences_BY_Round(data_directory_path)  
            
    DNAsOccurrences_IN_BaseRound = DNAs_BY_Round[base_cycle]
    BaseRoundSortedDNAsList = sorted(DNAsOccurrences_IN_BaseRound, key = DNAsOccurrences_IN_BaseRound.get, reverse = True)
    
    return BaseRoundSortedDNAsList

In [14]:
def DNAClonesOccurrences_BY_Round_BY_peptide(data_directory_path):
    display_summary = Completedisplay_summary(data_directory_path)
    
    DNAClonesOccurrences_BY_Round_BY_peptide = {}
    for Round in display_summary:
        DNAClonesOccurrences_BY_peptide = {}
        for peptide in display_summary[Round]:
            DNAClonesOccurrences_BY_peptide[peptide] = len(display_summary[Round][peptide])
        DNAClonesOccurrences_BY_Round_BY_peptide[Round] = DNAClonesOccurrences_BY_peptide
        
    return DNAClonesOccurrences_BY_Round_BY_peptide

In [15]:
def peptidesAppearances_BY_Round(BaseRoundSortedpeptidesList, peptidesOccurrences_BY_Round):
    
    peptidesAppearances_BY_Round = {}
    
    for peptide in BaseRoundSortedpeptidesList:
        peptidesAppearances_BY_Round[peptide] = []
        for Round in peptidesOccurrences_BY_Round:
            if peptide in peptidesOccurrences_BY_Round[Round]:
                peptidesAppearances_BY_Round[peptide] += [Round]
    return peptidesAppearances_BY_Round

In [16]:
def DNAsAppearances_BY_Round(BaseRoundSortedDNAsList, DNAsOccurrences_BY_Round):
    
    DNAsAppearances_BY_Round = {}
    
    for DNA in BaseRoundSortedDNAsList:
        DNAsAppearances_BY_Round[DNA] = []
        for Round in DNAsOccurrences_BY_Round:
            if DNA in DNAsOccurrences_BY_Round[Round]:
                DNAsAppearances_BY_Round[DNA] += [Round]
    return DNAsAppearances_BY_Round

In [17]:
def ComplementarySequence(String):
    DNAComplement = {'A':'T','C':'G','G':'C','T':'A'}
    RNAComplement = {'A':'U','C':'G','G':'C','U':'A'}
    
    ComplementarySequence = ''
    
    if 'T' in String:
        for i in range(len(String)):
            ComplementarySequence = ComplementarySequence + DNAComplement[String[i]]
    elif 'U' in String:
        for i in range(len(String)):
            ComplementarySequence = ComplementarySequence + RNAComplement[String[i]]
    return ComplementarySequence

In [18]:
def ReverseSequence(String):
    ReverseSequence = ''
    i = len(String) - 1
    while i >= 0:
        ReverseSequence = ReverseSequence + String[i]
        i -= 1
    return ReverseSequence

In [19]:
def display_summaryReport(data_directory_path, base_cycle, n_top_peptides, file_name):
    
    today = TodaysDate() 
    
    display_summaryFileNameCSV = str(today) + 'display_summary' + file_name + '.csv'
    display_summaryReportFile = open(display_summaryFileNameCSV, 'w')
    
    display_summary = Completedisplay_summary(data_directory_path)
    SortedRoundsList = sorted(display_summary.keys())
    
    peptides_BY_Round = peptidesOccurrences_BY_Round(data_directory_path)
    DNAs_BY_Round = DNAsOccurrences_BY_Round(data_directory_path)
    Totalpeptides_BY_Round = TotalReads_BY_Round(data_directory_path)

    BaseRoundSortedpeptides = BaseRoundSortedpeptidesList(data_directory_path, base_cycle)
    BaseRoundTopSortedpeptides = BaseRoundSortedpeptides[0 : (n_top_peptides)]
    
    #print ('total reads by round ' + str(Totalpeptides_BY_Round))
    #print (len(DNAs_BY_Round[base_cycle]))
    #print (DNAs_BY_Round[base_cycle].keys())
    BaseRoundcDNAs = list(DNAs_BY_Round[base_cycle].keys())
    
    NNKCodons = ['AAG', 'CCT', 'CCG', 'GAT',
                 'TGT', 'TTT', 'GAG', 'GCT',
                 'GCG', 'ACG', 'ACT', 'TAT',
                 'CTG', 'TTG', 'CTT', 'TAG',
                 'TGG', 'ATT', 'CGG', 'CGT',
                 'AGG', 'ATG', 'GGT', 'GGG',
                 'CAG', 'GTT', 'GTG', 'TCT',
                 'TCG', 'AGT', 'AAT', 'CAT']
    
    CodonsNumber = 0
    NonNNKCodonsNumber = 0
    
    NonNNKDictionary = {}
    
    for cDNA in BaseRoundcDNAs:
        OriginalcDNA = cDNA
        NonNNKScore = 0
        while len(cDNA) != 0:
            CodonsNumber += 1
            if cDNA[0:3] not in NNKCodons:
                NonNNKCodonsNumber += 1
                NonNNKScore += 1
            cDNA = cDNA[3:]
        NonNNKDictionary[OriginalcDNA] = NonNNKScore
                              
    #print (CodonsNumber)
    #print (NonNNKCodonsNumber)
    #print (NonNNKDictionary)
    
    NNKcDNAs = []
    for cDNA in NonNNKDictionary:
        if NonNNKDictionary[cDNA] == 0:
            NNKcDNAs += [cDNA]
    
    print (NNKcDNAs)
    print (len(NNKcDNAs))
    print ('GTTTGGGATCCGAGGACGTTTTATTTGTCTAGGATT' in NNKcDNAs)
    
    
    
    
    BaseRoundToppeptidecDNAs = display_summary[base_cycle][BaseRoundTopSortedpeptides[0]]
    print (len(BaseRoundToppeptidecDNAs))    
    
    ToppeptideClonesNonNNKDictionary = {}
    
    for cDNA in BaseRoundToppeptidecDNAs:
        OriginalcDNA = cDNA
        NonNNKScore = 0
        while len(cDNA) != 0:
            CodonsNumber += 1
            if cDNA[0:3] not in NNKCodons:
                NonNNKCodonsNumber += 1
                NonNNKScore += 1
            cDNA = cDNA[3:]
        ToppeptideClonesNonNNKDictionary[OriginalcDNA] = NonNNKScore
            
                    
    #print (CodonsNumber)
    #print (NonNNKCodonsNumber)
    #print (NonNNKDictionary)
    
    ToppeptideClonesNNKcDNAs = []
    for cDNA in ToppeptideClonesNonNNKDictionary:
        if ToppeptideClonesNonNNKDictionary[cDNA] == 0:
            ToppeptideClonesNNKcDNAs += [cDNA]
    
    print (ToppeptideClonesNNKcDNAs)
    print (len(ToppeptideClonesNNKcDNAs))
    print ('GTTTGGGATCCGAGGACGTTTTATTTGTCTAGGATT' in ToppeptideClonesNNKcDNAs)

In [20]:
data_directory_path = '/Users/nikita/Documents/DataSequensing/PHD2/SequencingResults/PHD2 NNK'
base_cycle = 6
TopNpeptidesNumber = 24
SummaryFileName = 'Paper05_PHD2SelectionResults'

display_summaryReport(data_directory_path, base_cycle, TopNpeptidesNumber, SummaryFileName)

82
['GTTTGGGATTCGAGGACGTTTTATTTGTCTAGGATT', 'GATAAGACTGGTGTGCTGTAGGAT', 'TGGGATCCTGCTACTATGTTTATTGCGCGTCGTTGT', 'GTTTGGGATCCTAGGACGTTTTATTTGTCTAGGATT', 'TGGGATCCTGCTACTATGTTTATTGTGCGTCGTTGT', 'TGGGATACTTCGACGGTTTATGTGCGGAGGCAG', 'GTTTGGGATACGAGGACGTTTTATTTGTCTAGGGTT', 'CGGACTTGGCCGTTGTATATTCCGATGTAT', 'TGGAATCCTCGGACTATTTTTATTAAGCGTGCT', 'GTTTGGGATCCGAGGACGTTTTATATGTCTAGGATT', 'GTTTGGGATCCGAGGACGTTTTATTTGTCTAGGAAT', 'TGGATTCATACTCATTATATTCATCGTACT', 'CAGCGTGATGTTGCGCCTTAGATTCGTGGTAATATG', 'ATTTGGGATACTGCTACTTTTTATCTGTCTAGGACT', 'GATGTTGTGAAGTTTGCTGTTATTGTGTGGTGTTAT', 'TCGTGGAATACTCGTTCGTTTTATCTTTCGCGGATT', 'TGGAATAGTTTTATTCTGTCTCGGGCTACTCATAAG', 'TGGTGGAATACTCGTTCGTTTTATTTGTCTAGGATT', 'GTTTGGGATCCGAGGACGTCTTATTTGTCTAGGATT', 'GTTCGTTTTCCGTTTTATATTCAGCGTAGG', 'GTTTGGGATCCGAGGACGTTTTATTTGTCTAGGTTT', 'TGGGATCCTGCTACTCTGTTTATTGTGCGTCGTTGT', 'GTTTGGGATCCGTGGACGTTTTATTTGTCTAGGATT', 'GTTCGGGATCCGAGGACGTTTTATTTGTCTAGGATT', 'ATTTGGGATACTGGTACTTTTCATCTGTCTAGGACT', 'GTTTGGGATCCGAGGACGTTTTATTAGTCTA

In [21]:
print(len('AATCCTAGACAAATAAAACGTCCTCGGATCCCAAAC'))

36


In [22]:
Translation = {'AAA':'K','AAC':'N','AAG':'K','AAU':'N',
                'ACA':'T','ACC':'T','ACG':'T','ACU':'T',
                'AGA':'R','AGC':'S','AGG':'R','AGU':'S',
                'AUA':'I','AUC':'I','AUG':'M','AUU':'I',
                
                'CAA':'Q','CAC':'H','CAG':'Q','CAU':'H',
                'CCA':'P','CCC':'P','CCG':'P','CCU':'P',
                'CGA':'R','CGC':'R','CGG':'R','CGU':'R',
                'CUA':'L','CUC':'L','CUG':'L','CUU':'L',
                
                'GAA':'E','GAC':'D','GAG':'E','GAU':'D',
                'GCA':'A','GCC':'A','GCG':'A','GCU':'A',
                'GGA':'G','GGC':'G','GGG':'G','GGU':'G',
                'GUA':'V','GUC':'V','GUG':'V','GUU':'V',
                
                'UAA':'ochre','UAC':'Y','UAG':'amber','UAU':'Y',
                'UCA':'S','UCC':'S','UCG':'S','UCU':'S',
                'UGA':'opal','UGC':'C','UGG':'W','UGU':'C',
                'UUA':'L','UUC':'F','UUG':'L','UUU':'F'}

ReverseTranslation = {}

for i in Translation:
    if Translation[i] not in ReverseTranslation:
        ReverseTranslation[Translation[i]] = [i]
    else:
        ReverseTranslation[Translation[i]] += [i]
        

print (ReverseTranslation)
print (ReverseTranslation['A'])
SortedReverseTranslation = sorted(ReverseTranslation)

NNKReverseTranslation = {}

for AminoAcid in SortedReverseTranslation:
    for Codon in ReverseTranslation[AminoAcid]:
        if Codon[2] == 'G' or Codon[2] == 'U':
            if AminoAcid not in NNKReverseTranslation:
                NNKReverseTranslation[AminoAcid] = [Codon]
            else:
                NNKReverseTranslation[AminoAcid] += [Codon]

print (NNKReverseTranslation)

NNKCodons = []
for AminoAcid in NNKReverseTranslation:
    NNKCodons += NNKReverseTranslation[AminoAcid]

print (NNKCodons)

{'I': ['AUA', 'AUU', 'AUC'], 'F': ['UUU', 'UUC'], 'H': ['CAU', 'CAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'opal': ['UGA'], 'V': ['GUC', 'GUG', 'GUA', 'GUU'], 'Y': ['UAC', 'UAU'], 'Q': ['CAA', 'CAG'], 'P': ['CCG', 'CCC', 'CCU', 'CCA'], 'A': ['GCU', 'GCG', 'GCA', 'GCC'], 'W': ['UGG'], 'ochre': ['UAA'], 'S': ['UCA', 'UCG', 'UCU', 'AGC', 'UCC', 'AGU'], 'E': ['GAG', 'GAA'], 'D': ['GAU', 'GAC'], 'C': ['UGU', 'UGC'], 'T': ['ACA', 'ACC', 'ACG', 'ACU'], 'amber': ['UAG'], 'R': ['AGA', 'CGU', 'AGG', 'CGG', 'CGA', 'CGC'], 'G': ['GGU', 'GGA', 'GGG', 'GGC'], 'N': ['AAC', 'AAU'], 'L': ['UUA', 'UUG', 'CUG', 'CUC', 'CUU', 'CUA']}
['GCU', 'GCG', 'GCA', 'GCC']
{'Y': ['UAU'], 'I': ['AUU'], 'F': ['UUU'], 'amber': ['UAG'], 'P': ['CCG', 'CCU'], 'A': ['GCU', 'GCG'], 'H': ['CAU'], 'V': ['GUG', 'GUU'], 'K': ['AAG'], 'S': ['UCG', 'UCU', 'AGU'], 'E': ['GAG'], 'D': ['GAU'], 'Q': ['CAG'], 'C': ['UGU'], 'T': ['ACG', 'ACU'], 'W': ['UGG'], 'M': ['AUG'], 'G': ['GGU', 'GGG'], 'N': ['AAU'], 'R': ['CGU', 'AGG', 'CGG'], 'L